#   **LangChain Custom Tool** (Part1)

- 사용자 정의 도구 (Custom Tool) 만들기

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

---

##  **사용자 정의 도구 (Custom Tool)**


- **사용자 정의 도구**는 개발자가 직접 설계하고 구현하는 **맞춤형 함수나 도구**를 의미

- LLM이 호출할 수 있는 **고유한 기능**을 정의하여 특정 작업에 최적화된 도구 생성 가능

- 개발자는 도구의 **입력값, 출력값, 기능**을 자유롭게 정의하여 유연한 확장성 확보

---

### 1. **`@tool` 데코레이터** 

- **@tool** 데코레이터는 **사용자 정의 도구**를 만드는 가장 기본적인 방법

- 함수의 **이름**과 **독스트링**을 자동으로 도구 정보로 활용

- 간단한 데코레이터 문법으로 **빠른 도구 개발** 가능

`(1) 기본 도구 만들기`

In [3]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

In [4]:
# DB 검색하는 사용자 정의 도구 생성
from langchain_core.tools import tool
from typing import Optional

@tool
def search_database(query: str, k: Optional[int] = 4) -> str:
    """
    데이터베이스에서 주어진 쿼리로 검색을 수행합니다.
    
    Args:
        query: 검색할 텍스트 쿼리
        k: 반환할 결과의 개수 (기본값: 4)
    """
    retriever = chroma_db.as_retriever(search_kwargs={"k": k})
    return retriever.invoke(query)

In [5]:
# 도구 속성 확인
print(search_database.name)  
print("-" * 100)
print(search_database.description) 
print("-" * 100)
print(search_database.args)  
print("-" * 100)
print(search_database.output_schema.model_json_schema())

search_database
----------------------------------------------------------------------------------------------------
데이터베이스에서 주어진 쿼리로 검색을 수행합니다.

Args:
    query: 검색할 텍스트 쿼리
    k: 반환할 결과의 개수 (기본값: 4)
----------------------------------------------------------------------------------------------------
{'query': {'title': 'Query', 'type': 'string'}, 'k': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': 4, 'title': 'K'}}
----------------------------------------------------------------------------------------------------
{'title': 'search_database_output'}


In [6]:
# 도구 실행 
docs = search_database.invoke("리비안은 언제 설립되었나요?")
pprint(docs)

[Document(id='ff588b7a-95d3-4949-aabb-fd080a25daa2', metadata={'language': 'ko', 'source': 'data/리비안_KR.md', 'company': '리비안'}, page_content='[출처] 이 문서는 리비안에 대한 문서입니다.\n----------------------------------\n- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- **설립:** 2009년 6월, 플로리다 주 록ledge\n- **설립자:** R. J. 스캐린지\n- **본사:** 미국 캘리포니아 주 어바인\n- **서비스 지역:** 북미\n- **주요 인물:** R. J. 스캐린지 (CEO)\n- **제품:** 전기 자동차, 배터리\n- **생산량 (2023):** 57,232대\n- **서비스:** 전기 자동차 충전, 자동차 보험\n- **수익 (2023):** 44억 3천만 미국 달러\n- **순이익 (2023):** -54억 미국 달러\n- **총 자산 (2023):** 168억 미국 달러'),
 Document(id='e9ebf1e5-ae4e-46b8-9c1b-a8c8b906fa91', metadata={'source': 'data/리비안_KR.md', 'language': 'ko', 'company': '리비안'}, page_content='[출처] 이 문서는 리비안에 대한 문서입니다.\n----------------------------------\nRivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.\n\n**주요 정보:**'),
 Document(id='b7357abb-85fb-4286-b9dd-9bb2fb75044a', metadata={'language': 'ko', 'source': 'data/리비안_KR.md', 'company': '리비안'}, page

In [7]:
# LLM 도구 바인딩하여 실행 
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)
llm_with_tools = llm.bind_tools([search_database])

# 도구 사용
result = llm_with_tools.invoke("리비안은 언제 설립되었나요?")

# 결과 확인

pprint(result.tool_calls)

[{'args': {'k': 1, 'query': '리비안 설립 연도'},
  'id': 'call_vvrzFPHiVvyS2hRyEyYt3PgJ',
  'name': 'search_database',
  'type': 'tool_call'}]


In [8]:
# 도구 사용 (k=2)
result = llm_with_tools.invoke("리비안은 언제 설립되었나요? (2개 문서 검색)")

pprint(result.tool_calls)

[{'args': {'query': '리비안 설립일'},
  'id': 'call_lO03XvLEnUe1EKcp3tpwFC33',
  'name': 'search_database',
  'type': 'tool_call'},
 {'args': {'query': '리비안 설립 연도'},
  'id': 'call_MloZDcMBa3Wz8UI2pkpoHU2j',
  'name': 'search_database',
  'type': 'tool_call'}]


`(2) 도구 이름 및 스키마 커스터마이징`

- **@tool 데코레이터**를 사용하여 도구의 속성을 직접 설정 가능
- 도구의 **이름**과 **스키마**를 개발자가 원하는 대로 커스터마이징할 수 있음

In [9]:
from pydantic import BaseModel, Field

# 도구 입력 스키마 정의 (pydantic 모델 사용)
class ChromaDBInput(BaseModel):
    """ ChromaDB 검색 도구 입력 스키마 """
    query: str = Field(description="검색할 쿼리")
    k: int = Field(4, description="반환할 문서의 개수")

@tool("ChromaDB-Search", args_schema=ChromaDBInput)
def search_database(query: str, k: int = 4) -> str:
    """
    데이터베이스에서 주어진 쿼리로 검색을 수행합니다.
    
    Args:
        query: 검색할 텍스트 쿼리
        k: 반환할 결과의 개수 (기본값: 4)
    """
    retriever = chroma_db.as_retriever(search_kwargs={"k": k})
    return retriever.invoke(query)

In [10]:
# 도구 속성 확인
print(search_database.name)  
print("-" * 100)
print(search_database.description) 
print("-" * 100)
print(search_database.args)  
print("-" * 100)
print(search_database.output_schema.model_json_schema())

ChromaDB-Search
----------------------------------------------------------------------------------------------------
데이터베이스에서 주어진 쿼리로 검색을 수행합니다.

Args:
    query: 검색할 텍스트 쿼리
    k: 반환할 결과의 개수 (기본값: 4)
----------------------------------------------------------------------------------------------------
{'query': {'description': '검색할 쿼리', 'title': 'Query', 'type': 'string'}, 'k': {'default': 4, 'description': '반환할 문서의 개수', 'title': 'K', 'type': 'integer'}}
----------------------------------------------------------------------------------------------------
{'title': 'ChromaDB-SearchOutput'}


---
### **[실습]**

- 도구 이름과 스키마를 직접 정의하여 도구를 생성합니다. 
- 도구 속성을 확인합니다. 
- 도구를 실행합니다. 
- LLM 모델에 도구를 바인딩하여 도구 호출 결과를 확인합니다. 

In [11]:
# 여기에 코드를 작성하세요. 

# 1. 도구 이름과 스키마를 직접 정의하여 도구 생성
from pydantic import BaseModel, Field
from langchain_core.tools import tool

# 입력 스키마 정의
class DatabaseSearchInput(BaseModel):
    """ 데이터베이스 검색 도구 입력 스키마 """
    query: str = Field(description="검색할 텍스트 쿼리")
    k: int = Field(3, description="반환할 문서의 개수 (기본값: 3)")

# 도구 생성
@tool("Database-Search-Tool", args_schema=DatabaseSearchInput)
def my_search_tool(query: str, k: int = 3) -> str:
    """
    ChromaDB 데이터베이스에서 주어진 쿼리로 문서를 검색합니다.
    
    Args:
        query: 검색할 텍스트 쿼리
        k: 반환할 결과의 개수 (기본값: 3)
    """
    retriever = chroma_db.as_retriever(search_kwargs={"k": k})
    return retriever.invoke(query)

# 2. 도구 속성 확인
print("도구 이름:", my_search_tool.name)
print("-" * 100)
print("도구 설명:", my_search_tool.description)
print("-" * 100)
print("도구 인자 스키마:", my_search_tool.args)
print("-" * 100)
print("도구 출력 스키마:", my_search_tool.output_schema.model_json_schema())
print("-" * 100)

# 3. 도구 실행
docs = my_search_tool.invoke({"query": "리비안은 언제 설립되었나요?", "k": 3})
print("\n도구 실행 결과:")
pprint(docs)
print("-" * 100)

# 4. LLM 모델에 도구를 바인딩하여 도구 호출 결과 확인
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
llm_with_tools = llm.bind_tools([my_search_tool])

# 도구 호출
result = llm_with_tools.invoke("리비안의 설립 연도를 알려주세요. (3개 문서 검색)")

print("\nLLM 도구 호출 결과:")
pprint(result.tool_calls)

도구 이름: Database-Search-Tool
----------------------------------------------------------------------------------------------------
도구 설명: ChromaDB 데이터베이스에서 주어진 쿼리로 문서를 검색합니다.

Args:
    query: 검색할 텍스트 쿼리
    k: 반환할 결과의 개수 (기본값: 3)
----------------------------------------------------------------------------------------------------
도구 인자 스키마: {'query': {'description': '검색할 텍스트 쿼리', 'title': 'Query', 'type': 'string'}, 'k': {'default': 3, 'description': '반환할 문서의 개수 (기본값: 3)', 'title': 'K', 'type': 'integer'}}
----------------------------------------------------------------------------------------------------
도구 출력 스키마: {'title': 'Database-Search-ToolOutput'}
----------------------------------------------------------------------------------------------------

도구 실행 결과:
[Document(id='ff588b7a-95d3-4949-aabb-fd080a25daa2', metadata={'company': '리비안', 'language': 'ko', 'source': 'data/리비안_KR.md'}, page_content='[출처] 이 문서는 리비안에 대한 문서입니다.\n----------------------------------\n- **회사 유형:** 상장\n

`(3) 비동기 도구 만들기`

- **비동기 도구**는 LangChain에서 `@tool` 데코레이터를 통해 구현 가능
- LangChain은 **동기와 비동기** 두 가지 도구 유형을 모두 지원함
- 비동기 도구는 더 효율적인 **병렬 처리**가 가능한 장점이 있음

In [12]:
from langchain_core.tools import tool

@tool
async def search_database(query: str, k: int = 4) -> str:
    """
    데이터베이스에서 주어진 쿼리로 검색을 수행합니다.
    
    Args:
        query: 검색할 텍스트 쿼리
        k: 반환할 결과의 개수 (기본값: 4)
    """
    retriever = chroma_db.as_retriever(search_kwargs={"k": k})
    return await retriever.ainvoke(query)

In [13]:
# 도구 속성 확인
print(search_database.name)
print("-" * 100)
print(search_database.description)
print("-" * 100)
print(search_database.args)
print("-" * 100)
print(search_database.output_schema.model_json_schema())

search_database
----------------------------------------------------------------------------------------------------
데이터베이스에서 주어진 쿼리로 검색을 수행합니다.

Args:
    query: 검색할 텍스트 쿼리
    k: 반환할 결과의 개수 (기본값: 4)
----------------------------------------------------------------------------------------------------
{'query': {'title': 'Query', 'type': 'string'}, 'k': {'default': 4, 'title': 'K', 'type': 'integer'}}
----------------------------------------------------------------------------------------------------
{'title': 'search_database_output'}


In [14]:
# 도구 실행 (비동기)

docs = await search_database.ainvoke("리비안은 언제 설립되었나요?")
pprint(docs)

[Document(id='ff588b7a-95d3-4949-aabb-fd080a25daa2', metadata={'language': 'ko', 'company': '리비안', 'source': 'data/리비안_KR.md'}, page_content='[출처] 이 문서는 리비안에 대한 문서입니다.\n----------------------------------\n- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- **설립:** 2009년 6월, 플로리다 주 록ledge\n- **설립자:** R. J. 스캐린지\n- **본사:** 미국 캘리포니아 주 어바인\n- **서비스 지역:** 북미\n- **주요 인물:** R. J. 스캐린지 (CEO)\n- **제품:** 전기 자동차, 배터리\n- **생산량 (2023):** 57,232대\n- **서비스:** 전기 자동차 충전, 자동차 보험\n- **수익 (2023):** 44억 3천만 미국 달러\n- **순이익 (2023):** -54억 미국 달러\n- **총 자산 (2023):** 168억 미국 달러'),
 Document(id='e9ebf1e5-ae4e-46b8-9c1b-a8c8b906fa91', metadata={'source': 'data/리비안_KR.md', 'language': 'ko', 'company': '리비안'}, page_content='[출처] 이 문서는 리비안에 대한 문서입니다.\n----------------------------------\nRivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.\n\n**주요 정보:**'),
 Document(id='b7357abb-85fb-4286-b9dd-9bb2fb75044a', metadata={'language': 'ko', 'company': '리비안', 'source': 'data/리비안_KR.md'}, page

---
### **[실습]**

- mmr 검색 리트리버를 사용하여 비동기 방식으로 동작하는 도구를 생성합니다. 
- 도구 속성을 확인합니다. 
- 도구를 실행합니다. 

In [15]:
# 여기에 코드를 작성하세요. 

# 1. MMR 검색 리트리버를 사용하여 비동기 방식으로 동작하는 도구 생성
from langchain_core.tools import tool

@tool
async def search_database_mmr(query: str, k: int = 4, fetch_k: int = 20) -> str:
    """
    MMR 검색을 사용하여 데이터베이스에서 다양성을 고려한 검색을 수행합니다.
    
    Args:
        query: 검색할 텍스트 쿼리
        k: 반환할 결과의 개수 (기본값: 4)
        fetch_k: MMR 알고리즘에 전달할 문서 개수 (기본값: 20)
    """
    retriever = chroma_db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": k, "fetch_k": fetch_k}
    )
    return await retriever.ainvoke(query)

# 2. 도구 속성 확인
print("도구 이름:", search_database_mmr.name)
print("-" * 100)
print("도구 설명:", search_database_mmr.description)
print("-" * 100)
print("도구 인자 스키마:", search_database_mmr.args)
print("-" * 100)
print("도구 출력 스키마:", search_database_mmr.output_schema.model_json_schema())
print("-" * 100)

# 3. 도구 실행 (비동기)
docs = await search_database_mmr.ainvoke({"query": "리비안은 언제 설립되었나요?", "k": 3, "fetch_k": 15})
print("\nMMR 검색 결과:")
pprint(docs)

도구 이름: search_database_mmr
----------------------------------------------------------------------------------------------------
도구 설명: MMR 검색을 사용하여 데이터베이스에서 다양성을 고려한 검색을 수행합니다.

Args:
    query: 검색할 텍스트 쿼리
    k: 반환할 결과의 개수 (기본값: 4)
    fetch_k: MMR 알고리즘에 전달할 문서 개수 (기본값: 20)
----------------------------------------------------------------------------------------------------
도구 인자 스키마: {'query': {'title': 'Query', 'type': 'string'}, 'k': {'default': 4, 'title': 'K', 'type': 'integer'}, 'fetch_k': {'default': 20, 'title': 'Fetch K', 'type': 'integer'}}
----------------------------------------------------------------------------------------------------
도구 출력 스키마: {'title': 'search_database_mmr_output'}
----------------------------------------------------------------------------------------------------

MMR 검색 결과:
[Document(id='ff588b7a-95d3-4949-aabb-fd080a25daa2', metadata={'company': '리비안', 'language': 'ko', 'source': 'data/리비안_KR.md'}, page_content='[출처] 이 문서는 리비안에 대한 문서입니다.\n-----

---

### 2. **StructuredTool** 

- **StructuredTool.from_function**을 통해 도구의 세부 동작을 정의할 수 있음

- 도구의 **실행 방식**과 **응답 처리**를 상세하게 커스터마이징 가능

- 개발자가 원하는 **특정 기능**을 도구에 쉽게 추가할 수 있음

`(1) 입출력 스키마 정의`

In [16]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 텍스트 분석 입력 스키마 정의
class TextAnalysisInput(BaseModel):
    text: str = Field(description="분석할 텍스트")
    include_sentiment: bool = Field(
        description="감성 분석 포함 여부", 
        default=False
    )

# 감성 분석 출력 스키마 정의
class SentimentOutput(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description="감성 분석 결과")

`(2) 도구 작업을 함수로 정의`

In [17]:
# 텍스트 분석 수행 함수 (동기)
def analyze_text(text: str, include_sentiment: bool = False) -> dict:
    """텍스트를 분석하여 단어 수, 문자 수 등의 정보를 반환합니다."""
    result = {
        "word_count": len(text.split()),
        "char_count": len(text),
        "sentence_count": len(text.split('.')),
    }
    
    if include_sentiment:
        # 감성 분석 수행
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "입력된 문장에 대해서 감성 분석을 수행합니다."),
                ("user", "{input}"),
            ]
        )
        llm =  ChatOpenAI(model="gpt-4.1-mini")

        llm_with_structure = llm.with_structured_output(SentimentOutput)

        sentiment_chain = prompt | llm_with_structure

        sentiment = sentiment_chain.invoke({"input": text})

        result["sentiment"] = sentiment.sentiment
        
    return result

# 텍스트 분석 수행 함수 (비동기)
async def analyze_text_async(text: str, include_sentiment: bool = False) -> dict:
    """텍스트 분석의 비동기 버전입니다."""
    return analyze_text(text, include_sentiment)

`(3) 도구 생성단계에서 커스터마이징 가능`

In [18]:
# 도구 생성
text_analyzer = StructuredTool.from_function(
    func=analyze_text,   # 동기 함수 사용
    name="TextAnalyzer",    # 도구 이름
    description="텍스트의 기본 통계와 선택적으로 감성 분석을 수행합니다.",   # 도구 설명
    args_schema=TextAnalysisInput,   # 입력 스키마
    coroutine=analyze_text_async,    # 비동기 함수 사용
    return_direct=True    # 결과를 직접 반환
)

# 도구 속성 확인
print(text_analyzer.name)
print(text_analyzer.description)
print(text_analyzer.args)
print(text_analyzer.output_schema.model_json_schema())

TextAnalyzer
텍스트의 기본 통계와 선택적으로 감성 분석을 수행합니다.
{'text': {'description': '분석할 텍스트', 'title': 'Text', 'type': 'string'}, 'include_sentiment': {'default': False, 'description': '감성 분석 포함 여부', 'title': 'Include Sentiment', 'type': 'boolean'}}
{'title': 'TextAnalyzerOutput'}


`(4) 도구 실행`

In [19]:
# 텍스트 분석 도구 사용
text = "안녕하세요. 오늘은 날씨가 좋네요. 산책하기 좋은 날입니다."

# 동기 호출
result1 = text_analyzer.invoke({
    "text": text,
    "include_sentiment": True
})
print("텍스트 분석 결과:", result1)

# 비동기 호출
result2 = await text_analyzer.ainvoke({
    "text": text,
    "include_sentiment": False
})
print("비동기 텍스트 분석 결과:", result2)

텍스트 분석 결과: {'word_count': 7, 'char_count': 33, 'sentence_count': 4, 'sentiment': 'positive'}
비동기 텍스트 분석 결과: {'word_count': 7, 'char_count': 33, 'sentence_count': 4}


`(5) StructuredTool은 다음과 같은 상황에서 더 적합 (@tool 데코레이터와 차이점)`

- **StructuredTool**은 **기존 함수**를 도구로 쉽게 변환하여 재활용 가능
- 하나의 함수로 **다양한 설정**의 도구를 만들 수 있어 코드 중복을 방지함
- **동기/비동기** 버전을 동시에 지원하여 유연한 실행 환경 제공

In [20]:
### 1. 기존 함수의 재사용

# 이미 존재하는 함수를 도구로 변환할 때
def existing_function(x: int) -> str:
    return str(x)

# @tool을 사용하려면 함수를 수정해야 함
from langchain_core.tools import tool

@tool 
def modified_function(x: int) -> str:
    """ 숫자 변환 도구 """
    return str(x)

# StructuredTool은 기존 함수를 그대로 사용 가능
tool = StructuredTool.from_function(
    func=existing_function,
    name="convert_number",
    description="숫자를 문자열로 변환합니다.",
)

In [21]:
### 2. 동일한 함수에 대해 다른 설정의 도구 생성

def multiply(a: int, b: int) -> int:
    return a * b

# 같은 함수로 다른 설정의 도구들을 만들 수 있음
basic_calculator = StructuredTool.from_function(
    func=multiply,
    name="basic_multiply",
    description="기본 곱셈 계산기",
)

advanced_calculator = StructuredTool.from_function(
    func=multiply,
    name="output_multiply", 
    description="결과 출력용",
    return_direct=True   # 결과를 직접 반환
)

# 도구 속성 확인
print(basic_calculator.name)
print(basic_calculator.description)
print(basic_calculator.args)
print(basic_calculator.output_schema.model_json_schema())
print()
print(advanced_calculator.name)
print(advanced_calculator.description)
print(advanced_calculator.args)
print(advanced_calculator.output_schema.model_json_schema())

basic_multiply
기본 곱셈 계산기
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'title': 'basic_multiply_output'}

output_multiply
결과 출력용
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'title': 'output_multiply_output'}


In [22]:
from langchain.agents import create_agent

# 도구 실행 에이전트 생성 (return_direct=False)
basic_agent = create_agent(
    model=llm,
    tools=[basic_calculator],
    system_prompt="당신은 수학 계산을 도와주는 AI 어시스턴트입니다."
)

# 도구 실행 에이전트 사용
result = basic_agent.invoke(
    {"messages": [{"role": "user", "content": "2와 3을 곱해줘"}]},
)

pprint(result["messages"])

[HumanMessage(content='2와 3을 곱해줘', additional_kwargs={}, response_metadata={}, id='f63dba9f-ec0d-4343-b059-b26acb51213b'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 77, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CTRAuCrK4Vu7Cppx5VluoWao2OLgz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f0b87685-5cce-48af-84d1-e6a6a87ec543-0', tool_calls=[{'name': 'basic_multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_ZVTth6347hGxB0sBhWbJyCTL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 77, 'output_tokens': 19, 'total_tokens': 96, 'input_token_details'

In [23]:
# 도구 실행 에이전트 생성 (return_direct=True)
advanced_agent = create_agent(
    model=llm,
    tools=[advanced_calculator],
    system_prompt="당신은 수학 계산을 도와주는 AI 어시스턴트입니다."
)

# 도구 실행 에이전트 사용
result = advanced_agent.invoke(
    {"messages": [{"role": "user", "content": "2와 3을 곱해줘"}]},
)

pprint(result["messages"])

[HumanMessage(content='2와 3을 곱해줘', additional_kwargs={}, response_metadata={}, id='fe651b03-8def-4e51-8d41-7563ec50fe80'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 73, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CTRAwbbiSnl2KhRQWXgvEzgkueHtp', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4f4b7d94-0d50-43f5-b254-507dcb02db4a-0', tool_calls=[{'name': 'output_multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_ApYELn8WHbQ1KryUykhsPO0R', 'type': 'tool_call'}], usage_metadata={'input_tokens': 73, 'output_tokens': 19, 'total_tokens': 92, 'input_token_details

In [24]:
### 3. 동기/비동기 함수 동시 지원

def sync_func(x: int) -> int:
    return x * 2

async def async_func(x: int) -> int:
    return sync_func(x)

# 동기/비동기 함수를 하나의 도구로 결합
tool = StructuredTool.from_function(
    func=sync_func,  # 동기 함수
    coroutine=async_func,  # 비동기 함수
    name="multiply_by_2",
    description="입력된 숫자에 2를 곱합니다."
)

# 사용
result1 = tool.invoke({"x": 5})  # 동기 호출
result2 = await tool.ainvoke({"x": 5})  # 비동기 호출

print(result1)
print(result2)

10
10


---
### **[실습]**

- StructuredTool을 이용하여 mmr 검색 리트리버를 사용하는 도구를 생성합니다. 
- 도구 속성을 확인합니다. 
- 도구를 실행합니다. 

In [25]:
# 여기에 코드를 작성하세요. 

from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

# 1. 입력 스키마 정의
class MMRSearchInput(BaseModel):
    """MMR 검색 도구 입력 스키마"""
    query: str = Field(description="검색할 쿼리")
    k: int = Field(4, description="반환할 문서의 개수")
    fetch_k: int = Field(20, description="MMR 알고리즘에 전달할 문서 개수")

# 2. MMR 검색 수행 함수 (동기)
def mmr_search(query: str, k: int = 4, fetch_k: int = 20) -> list:
    """MMR 검색을 수행하여 다양성을 고려한 문서를 반환합니다."""
    retriever = chroma_db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": k, "fetch_k": fetch_k}
    )
    return retriever.invoke(query)

# 3. MMR 검색 수행 함수 (비동기)
async def mmr_search_async(query: str, k: int = 4, fetch_k: int = 20) -> list:
    """MMR 검색의 비동기 버전입니다."""
    retriever = chroma_db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": k, "fetch_k": fetch_k}
    )
    return await retriever.ainvoke(query)

# 4. StructuredTool로 도구 생성
mmr_search_tool = StructuredTool.from_function(
    func=mmr_search,
    name="MMR_Search",
    description="MMR 알고리즘을 사용하여 다양성을 고려한 문서 검색을 수행합니다.",
    args_schema=MMRSearchInput,
    coroutine=mmr_search_async,
    return_direct=False
)

# 5. 도구 속성 확인
print("도구 이름:", mmr_search_tool.name)
print("-" * 100)
print("도구 설명:", mmr_search_tool.description)
print("-" * 100)
print("도구 인자 스키마:", mmr_search_tool.args)
print("-" * 100)
print("도구 출력 스키마:", mmr_search_tool.output_schema.model_json_schema())
print("-" * 100)

# 6. 도구 실행 (동기)
result_sync = mmr_search_tool.invoke({
    "query": "리비안의 전기 트럭에 대해 알려주세요",
    "k": 3,
    "fetch_k": 15
})
print("\n동기 MMR 검색 결과:")
pprint(result_sync)
print("-" * 100)

# 7. 도구 실행 (비동기)
result_async = await mmr_search_tool.ainvoke({
    "query": "테슬라의 전기차 기술",
    "k": 3,
    "fetch_k": 15
})
print("\n비동기 MMR 검색 결과:")
pprint(result_async)

도구 이름: MMR_Search
----------------------------------------------------------------------------------------------------
도구 설명: MMR 알고리즘을 사용하여 다양성을 고려한 문서 검색을 수행합니다.
----------------------------------------------------------------------------------------------------
도구 인자 스키마: {'query': {'description': '검색할 쿼리', 'title': 'Query', 'type': 'string'}, 'k': {'default': 4, 'description': '반환할 문서의 개수', 'title': 'K', 'type': 'integer'}, 'fetch_k': {'default': 20, 'description': 'MMR 알고리즘에 전달할 문서 개수', 'title': 'Fetch K', 'type': 'integer'}}
----------------------------------------------------------------------------------------------------
도구 출력 스키마: {'title': 'MMR_SearchOutput'}
----------------------------------------------------------------------------------------------------

동기 MMR 검색 결과:
[Document(id='e9ebf1e5-ae4e-46b8-9c1b-a8c8b906fa91', metadata={'company': '리비안', 'language': 'ko', 'source': 'data/리비안_KR.md'}, page_content='[출처] 이 문서는 리비안에 대한 문서입니다.\n--------------------------------

---

### 3. **Runnable을 도구로 변환** 

- `as_tool` 메소드: **Runnable**을 도구로 변환하여 **복잡한 체인**을 하나의 단위로 관리 가능

- 도구화를 통해 체인에 대한 **명확한 인터페이스**를 제공

- 변환된 도구는 다른 프로젝트나 컴포넌트에서 쉽게 **재사용** 가능

- 체인의 실행 방식을 **표준화**하여 일관된 사용 경험 제공

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 이메일 작성 체인
email_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 전문적인 이메일 작성 도우미입니다."),
    ("human", """
    다음 정보로 이메일을 작성해주세요:
    - 수신자: {recipient}
    - 제목: {subject}
    - 톤: {tone}
    - 추가 요청사항: {requirements}
    """)
])

email_chain = (
    email_prompt 
    | ChatOpenAI(model="gpt-4.1-mini", temperature=0.7) 
    | StrOutputParser()
)

# 이메일 작성 도구로 변환
email_tool = email_chain.as_tool(
    name="email_writer",
    description="전문적인 이메일 작성을 도와주는 도구입니다.",
)

# 도구 속성 변경
email_tool.return_direct = True

# 도구 속성 확인
print(email_tool.name)
print(email_tool.description)
print(email_tool.args)
print(email_tool.return_direct)

email_writer
전문적인 이메일 작성을 도와주는 도구입니다.
{'recipient': {'title': 'Recipient', 'type': 'string'}, 'requirements': {'title': 'Requirements', 'type': 'string'}, 'subject': {'title': 'Subject', 'type': 'string'}, 'tone': {'title': 'Tone', 'type': 'string'}}
True


C:\Users\kaydash\AppData\Local\Temp\ipykernel_17976\3343907101.py:24: LangChainBetaWarning: This API is in beta and may change in the future.
  email_tool = email_chain.as_tool(


In [27]:
# 도구 실행
email_result = email_tool.invoke({
    "recipient": "team@example.com",
    "subject": "프로젝트 진행 현황 보고",
    "tone": "전문적",
    "requirements": "회의 일정 조율 요청 포함"
})

print(email_result)

Subject: 프로젝트 진행 현황 보고 및 회의 일정 조율 요청

team@example.com 귀하,

안녕하세요.

현재 진행 중인 프로젝트의 현황을 아래와 같이 보고드립니다.

[프로젝트 진행 현황 요약]
- 주요 완료 사항:
- 진행 중인 작업:
- 예상 일정 및 향후 계획:

추가 논의가 필요한 사항이 있어 회의 일정을 조율하고자 합니다. 가능한 회의 일정을 제안해 주시면 감사하겠습니다.

바쁘시겠지만 빠른 회신 부탁드립니다.

감사합니다.

[발신자 이름 드림]


In [28]:
# LLM과 도구 바인딩
llm_with_tools = llm.bind_tools([email_tool])
result = llm_with_tools.invoke("팀에게 프로젝트 진행 현황을 보고하는 이메일을 작성해줘. (전문적 톤, 요구사항: 회의 일정 조율 요청 포함, 수신자: 'team@email.com')")

pprint(result.tool_calls)


[{'args': {'recipient': 'team@email.com',
           'requirements': '회의 일정 조율 요청 포함',
           'subject': '프로젝트 진행 현황 보고',
           'tone': '전문적'},
  'id': 'call_45n3kqv7OoVLNq6D6PkuNUzU',
  'name': 'email_writer',
  'type': 'tool_call'}]


In [29]:
# 도구 호출 결과를 실행
tool_msg = email_tool.invoke(result.tool_calls[0])
print(tool_msg.content)

제목: 프로젝트 진행 현황 보고 및 회의 일정 조율 요청

안녕하세요 팀 여러분,

현재 진행 중인 프로젝트의 현황을 아래와 같이 보고드립니다.

[프로젝트 진행 현황 요약]
- 주요 완료 사항:
- 진행 중인 작업:
- 예상 일정 및 향후 계획:

추가로, 프로젝트 관련 논의를 위해 회의 일정을 조율하고자 합니다. 가능한 일정과 시간을 공유해주시면 감사하겠습니다.

감사합니다.

[당신의 이름]  
[당신의 직책]  
[연락처]


In [30]:
from langchain.agents import create_agent

# 도구 실행 에이전트 생성 
email_agent = create_agent(
    model=llm,
    tools=[email_tool],
    system_prompt="당신은 이메일 작성을 도와주는 AI 어시스턴트입니다."
)

# 도구 실행 에이전트 사용
result = email_agent.invoke(
    {"messages": [{"role": "user", "content": "팀에게 프로젝트 진행 현황을 보고하는 이메일을 작성해줘. (전문적 톤, 요구사항: 회의 일정 조율 요청 포함, 수신자: 'team@example.com')"}]},
)

pprint(result["messages"])

[HumanMessage(content="팀에게 프로젝트 진행 현황을 보고하는 이메일을 작성해줘. (전문적 톤, 요구사항: 회의 일정 조율 요청 포함, 수신자: 'team@example.com')", additional_kwargs={}, response_metadata={}, id='b4992d19-e723-4f24-98b1-027c7d16a04b'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 120, 'total_tokens': 166, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CTRB7di6xmjVLIleo7C7x04XnrWdq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--a5a67e84-8c75-4e05-8db9-a8a8fd479068-0', tool_calls=[{'name': 'email_writer', 'args': {'recipient': 'team@example.com', 'requirements': '회의 일정 조율 요청 포함', 'subject': '프로젝트 진행 현황 보고 및 회의 일정

---

### **[실습]**

- 검색 결과(문서)와 쿼리 간의 유사도를 CrossEncoderReranker를 사용하여 Re-rank 수행 (k:10 -> top_n:3)
- 검색 결과를 포맷팅하여 출력하는 Runnable 체인을 구성
- Runnable 체인을 도구로 변환 

In [31]:
# 여기에 코드를 작성하세요. 

# LangChain 1.0 호환 구현 - Cross-Encoder Reranker 직접 구현
from langchain_core.runnables import RunnableLambda
from sentence_transformers import CrossEncoder

# 1. Cross-Encoder 모델 초기화 및 Reranking 함수 구현
def search_and_rerank(query_input, k: int = 10, top_n: int = 3):
    """
    데이터베이스에서 k개 문서를 검색한 후 Cross-Encoder로 상위 top_n개를 선택합니다.
    
    Args:
        query_input: 검색 쿼리 (문자열 또는 딕셔너리)
        k: 초기 검색할 문서 개수 (기본값: 10)
        top_n: 최종 반환할 문서 개수 (기본값: 3)
    """
    # 입력이 딕셔너리인 경우 query 키에서 추출, 아니면 그대로 사용
    query = query_input["query"] if isinstance(query_input, dict) else query_input
    
    # Cross-Encoder 모델 초기화
    model = CrossEncoder("BAAI/bge-reranker-base")
    
    # 기본 retriever로 k개 문서 검색
    retriever = chroma_db.as_retriever(search_kwargs={"k": k})
    docs = retriever.invoke(query)
    
    # Cross-Encoder로 relevance score 계산
    pairs = [[query, doc.page_content] for doc in docs]
    scores = model.predict(pairs)
    
    # Score 기준으로 정렬하여 상위 top_n개 선택
    scored_docs = list(zip(docs, scores))
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    top_docs = [doc for doc, score in scored_docs[:top_n]]
    
    return top_docs

# 2. 검색 결과를 포맷팅하여 출력하는 함수
def format_docs(docs):
    """검색된 문서를 포맷팅하여 반환합니다."""
    formatted_result = f"검색 결과 ({len(docs)}개 문서):\n" + "=" * 100 + "\n\n"
    
    for idx, doc in enumerate(docs, 1):
        formatted_result += f"[문서 {idx}]\n"
        formatted_result += f"내용: {doc.page_content}\n"
        
        # 메타데이터가 있으면 출력
        if doc.metadata:
            formatted_result += f"메타데이터: {doc.metadata}\n"
        
        formatted_result += "-" * 100 + "\n\n"
    
    return formatted_result

# 3. Runnable 체인 구성
search_and_rerank_chain = (
    RunnableLambda(search_and_rerank)
    | RunnableLambda(format_docs)
)

# 4. Runnable 체인을 도구로 변환
search_rerank_tool = search_and_rerank_chain.as_tool(
    name="search_and_rerank",
    description="데이터베이스에서 10개 문서를 검색한 후 CrossEncoder를 사용하여 상위 3개를 선택하여 포맷팅된 결과를 반환합니다."
)

# 도구 속성 확인
print("도구 이름:", search_rerank_tool.name)
print("-" * 100)
print("도구 설명:", search_rerank_tool.description)
print("-" * 100)
print("도구 인자 스키마:", search_rerank_tool.args)
print("-" * 100)

# 도구 실행
result = search_rerank_tool.invoke({"query": "리비안의 전기 트럭 기술에 대해 알려주세요"})
print("\n검색 및 Re-rank 결과:")
print(result)

도구 이름: search_and_rerank
----------------------------------------------------------------------------------------------------
도구 설명: 데이터베이스에서 10개 문서를 검색한 후 CrossEncoder를 사용하여 상위 3개를 선택하여 포맷팅된 결과를 반환합니다.
----------------------------------------------------------------------------------------------------
도구 인자 스키마: {'query': {'title': 'Query'}}
----------------------------------------------------------------------------------------------------

검색 및 Re-rank 결과:
검색 결과 (3개 문서):

[문서 1]
내용: [출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**Volkswagen과의 파트너십 (2024)**

- 2024년 6월, Volkswagen Group은 전기 아키텍처 및 소프트웨어 기술 개발을 목표로 Rivian에 최대 50억 달러를 투자할 의향을 발표.

**차량**

- **R1T:** 4개의 전기 모터가 장착된 픽업 트럭. 배터리 크기는 105 kWh에서 180 kWh까지 다양함.
- **R1S:** 첫 번째 Rivian 플랫폼의 스포츠 유틸리티 차량(SUV) 버전.
- **Electric Delivery Van (EDV):** 상업용 전기 밴으로, 주로 Amazon용으로 설계되어 사용됨.
- **R2:** 더 작고 저렴한 SUV로, 새로운 플랫폼에서 2026년 초에 출시될 예정.
- **R3:** 출시 예정인 전기 소형 SUV.
메타데이터: {'company': '리비안', 'source': 'data/리비안_KR.md'